In [1]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
# import csv
# Set random seed for reproducibility
import random
random.seed(42)
np.random.seed(42)

# Define the new data for 30 documents with category and tags
# newData = [
#     {
#         "author": "Emily Johnson",
#         "claps": "1.5K",
#         "reading_time": "2 mins",
#         "link": "https://example.com/cities-article-1",
#         "title": "The Rise of Smart Cities",
#         "text": "Smart cities integrate technology to enhance the quality of urban life. By utilizing data and IoT devices, these cities improve infrastructure, optimize traffic flow, and reduce energy consumption. The rise of smart cities is transforming urban environments, making them more efficient, sustainable, and livable. As technology continues to advance, the potential for smarter, more connected cities grows, promising a better future for urban dwellers.",
#         "category": "World",
#         "tags": ["cities"]
#     },
#     {
#         "author": "Michael Smith",
#         "claps": "1.7K",
#         "reading_time": "2 mins",
#         "link": "https://example.com/cities-article-2",
#         "title": "Urbanization and Its Challenges",
#         "text": "Urbanization is a global phenomenon with significant impacts on society and the environment. As more people move to cities, challenges such as overcrowding, pollution, and inadequate infrastructure arise. Addressing these issues requires innovative solutions and sustainable planning. By focusing on smart growth and community development, cities can become more resilient and better equipped to meet the needs of their growing populations.",
#         "category": "World",
#         "tags": ["cities"]
#     },
#     {
#         "author": "Sophia Brown",
#         "claps": "1.9K",
#         "reading_time": "2 mins",
#         "link": "https://example.com/cities-article-3",
#         "title": "The Future of Urban Transportation",
#         "text": "The future of urban transportation is evolving with advancements in technology and infrastructure. Innovations such as electric vehicles, autonomous cars, and efficient public transit systems are transforming how we navigate cities. These developments aim to reduce traffic congestion, lower emissions, and improve accessibility. As cities embrace these changes, the potential for a more sustainable and convenient urban transportation system grows.",
#         "category": "World",
#         "tags": ["cities"]
#     },
#     {
#         "author": "James Williams",
#         "claps": "2.0K",
#         "reading_time": "2 mins",
#         "link": "https://example.com/nature-article-1",
#         "title": "The Importance of Biodiversity",
#         "text": "Biodiversity is crucial for maintaining the balance of ecosystems and ensuring the survival of various species. It provides essential services such as pollination, nutrient cycling, and climate regulation. Protecting biodiversity is vital for the health of our planet and the well-being of future generations. By conserving natural habitats and promoting sustainable practices, we can help preserve the rich diversity of life on Earth.",
#         "category": "World",
#         "tags": ["nature"]
#     },
#     {
#         "author": "Olivia Taylor",
#         "claps": "2.1K",
#         "reading_time": "2 mins",
#         "link": "https://example.com/nature-article-2",
#         "title": "Exploring the Wonders of National Parks",
#         "text": "National parks offer a glimpse into the natural beauty and biodiversity of our planet. They provide opportunities for recreation, education, and conservation. Visiting national parks allows us to connect with nature, learn about different ecosystems, and appreciate the importance of preserving these natural treasures. By supporting national parks, we contribute to the protection of wildlife and the conservation of our natural heritage.",
#         "category": "World",
#         "tags": ["nature"]
#     },
#     {
#         "author": "Isabella Moore",
#         "claps": "2.3K",
#         "reading_time": "2 mins",
#         "link": "https://example.com/nature-article-3",
#         "title": "The Role of Forests in Climate Regulation",
#         "text": "Forests play a vital role in regulating the Earth's climate. They act as carbon sinks, absorbing carbon dioxide from the atmosphere and storing it in biomass. Forests also influence weather patterns, protect watersheds, and support biodiversity. Deforestation and forest degradation pose significant threats to these critical functions. Protecting and restoring forests is essential for mitigating climate change and ensuring a sustainable future.",
#         "category": "World",
#         "tags": ["nature"]
#     },
#     {
#         "author": "Liam Martinez",
#         "claps": "2.4K",
#         "reading_time": "2 mins",
#         "link": "https://example.com/climate-change-article-1",
#         "title": "The Impact of Climate Change on Coastal Cities",
#         "text": "Climate change poses significant risks to coastal cities around the world. Rising sea levels, increased storm intensity, and coastal erosion threaten infrastructure, economies, and communities. Addressing these challenges requires innovative solutions such as resilient infrastructure, sustainable urban planning, and climate adaptation strategies. By taking proactive measures, we can reduce the vulnerability of coastal cities and protect their inhabitants.",
#         "category": "World",
#         "tags": ["climate change"]
#     },
#     {
#         "author": "Ava Rodriguez",
#         "claps": "2.5K",
#         "reading_time": "2 mins",
#         "link": "https://example.com/climate-change-article-2",
#         "title": "Renewable Energy: A Key to Combating Climate Change",
#         "text": "Renewable energy sources, such as solar, wind, and hydropower, are essential in the fight against climate change. They provide clean, sustainable alternatives to fossil fuels, reducing greenhouse gas emissions and dependence on non-renewable resources. Investing in renewable energy technologies and infrastructure is crucial for transitioning to a low-carbon economy and achieving global climate goals. Embracing renewable energy can lead to a more sustainable and resilient future.",
#         "category": "World",
#         "tags": ["climate change"]
#     },
#     {
#         "author": "Ethan Harris",
#         "claps": "2.7K",
#         "reading_time": "2 mins",
#         "link": "https://example.com/climate-change-article-3",
#         "title": "The Role of Individuals in Mitigating Climate Change",
#         "text": "Individuals play a crucial role in mitigating climate change through everyday actions and lifestyle choices. Reducing energy consumption, minimizing waste, and supporting sustainable products are effective ways to lower one's carbon footprint. Additionally, advocating for environmental policies and participating in community initiatives can amplify individual efforts. Collective action and personal responsibility are key to addressing the global challenge of climate change.",
#         "category": "World",
#         "tags": ["climate change"]
#     },
#     {
#         "author": "Mason Clark",
#         "claps": "2.8K",
#         "reading_time": "2 mins",
#         "link": "https://example.com/travel-article-1",
#         "title": "Sustainable Travel: Tips for Eco-Friendly Adventures",
#         "text": "Sustainable travel focuses on minimizing the environmental impact of tourism. Simple actions such as choosing eco-friendly accommodations, reducing plastic use, and respecting local cultures can make a significant difference. By supporting sustainable tourism practices, travelers can enjoy their adventures while preserving the natural beauty and cultural heritage of destinations. Responsible travel ensures that future generations can also experience the wonders of our planet.",
#         "category": "World",
#         "tags": ["travel"]
#     },
#     {
#         "author": "Charlotte Lee",
#         "claps": "3.0K",
#         "reading_time": "2 mins",
#         "link": "https://example.com/travel-article-2",
#         "title": "Exploring Off-the-Beaten-Path Destinations",
#         "text": "Traveling to off-the-beaten-path destinations offers unique experiences and helps reduce the pressures on popular tourist spots. These less-explored areas often provide a more authentic and immersive travel experience. By venturing off the usual tourist trails, travelers can discover hidden gems, connect with local communities, and contribute to the preservation of less-known cultural and natural attractions.",
#         "category": "World",
#         "tags": ["travel"]
#     },
#     {
#         "author": "Amelia Perez",
#         "claps": "3.2K",
#         "reading_time": "2 mins",
#         "link": "https://example.com/travel-article-3",
#         "title": "Cultural Immersion Through Travel",
#         "text": "Traveling offers an opportunity to immerse oneself in different cultures and broaden one's perspective. By engaging with local customs, trying traditional foods, and participating in cultural activities, travelers can gain a deeper understanding of the places they visit. Cultural immersion fosters empathy, respect, and appreciation for diversity, enriching the travel experience and contributing to global understanding and harmony.",
#         "category": "World",
#         "tags": ["travel"]
#     }
# ]



# # Path to the CSV file
# csv_file_path = 'articles.csv'

# # Read the existing data from the CSV file
# existing_data = []
# with open(csv_file_path, mode='r', newline='', encoding='utf-8') as file:
#     reader = csv.DictReader(file)
#     for row in reader:
#         existing_data.append(row)

# # Append the new data to the existing data
# existing_data.extend(newData)

# # Write all the data back to the CSV file
# with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
#     writer = csv.DictWriter(file, fieldnames=existing_data[0].keys())
#     writer.writeheader()
#     writer.writerows(existing_data)

# print(f"10 food articles have been written to {csv_file_path}")



# Load the CSV file
df = pd.read_csv('articlesFinal.csv')  # Update the filename as necessary

# Combine article title, author, and text into one column for processing
df['content'] = df['title'] + ' ' + df['author'] + ' ' + df['text']

# Drop unnecessary columns for processing, but keep original columns for output
df_proc = df.drop(['title', 'author', 'text', 'link', 'category', 'tags'], axis=1)

# Drop duplicate texts and reset index
df_proc.drop_duplicates(inplace=True)
df_proc.reset_index(drop=True, inplace=True)

# Initialize the lemmatizer and download necessary NLTK resources
lemma = WordNetLemmatizer()
nltk.download('stopwords')
stopword_list = stopwords.words('english')
nltk.download('punkt')
nltk.download('wordnet')

# Define a function for preprocessing the text
def clean(text):
    # Remove special characters
    text = re.sub("[^A-Za-z0-9 ]", " ", text)
    # Convert text to lowercase
    text = text.lower()
    # Tokenize text
    tokens = nltk.word_tokenize(text)
    # Remove stopwords and apply lemmatization
    text_list = [lemma.lemmatize(token) for token in tokens if token not in stopword_list]
    return ' '.join(text_list)

# Apply the cleaning function to the text column
df_proc['content'] = df_proc['content'].apply(clean)

# Compute a list of the texts
text_list = [list(i.split(' ')) for i in df_proc['content']]

# Tag the documents
tagged_data = [TaggedDocument(words=doc, tags=[i]) for i, doc in enumerate(text_list)]

# Train the Doc2Vec model with a fixed random seed
model = Doc2Vec(tagged_data, vector_size=5, window=2, min_count=1, workers=4, seed=42)

# Save the model
model.save("article_recommendation_model.d2v")

# Define a function to infer vector and get similar articles
def recommend_articles(input_text, topn=10):
    input_clean = clean(input_text)
    input_vectorized = model.infer_vector(input_clean.split(' '))
    similar_articles = model.dv.most_similar(positive=[input_vectorized], topn=topn)
    
    sim_list = []
    for ind, value in enumerate(similar_articles):
        index = value[0]
        sim_list.append([ind + 1, index, df['title'][index]]) #, df['text'][index], value[1]
    return pd.DataFrame(sim_list, columns=["rank", "original_index", "title"]) #text cos_sim

# Example usage
user_interest = "food"  # Example user interest
recommended_blogs = recommend_articles(user_interest)
print(recommended_blogs)


[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading wordnet: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


   rank  original_index                                              title
0     1               1             Understanding Children's Dietary Needs
1     2               6               Romantic Dinner Ideas for Date Night
2     3              16        Brunch with Friends: Menu Ideas and Recipes
3     4              22            Understanding the Gut-Health Connection
4     5              65                    Guided Meditation for Beginners
5     6              29  Healthy Snacks for Kids: Tasty and Nutritious ...
6     7              26                  Healthy Eating Habits for Couples
7     8              14       Cooking Classes for Couples: A Fun Date Idea
8     9             100  Introduction to iOS App Development: Getting S...
9    10               4            Understanding the Gut-Health Connection
